# Day 1 - Community Contribution by Hayatu

Website summarizer using OpenAI + a commercial use-case: generating email subject lines from email content.

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join("..", "..")))

from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

openai = OpenAI()

## Website Summarizer

Scrape a website and summarize its contents using GPT with a snarky tone.

In [ ]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://hayatusanusi.xyz")

## Commercial Use-Case: Email Subject Line Generator

Given the full body of an email, generate a concise, informative subject line.

In [ ]:
system_prompt = "Your job is to generate a short name from a given email message. The name should tell me what i need to know about the email."
user_prompt = """
    Personio: Update on your Application | Software Engineer - Front End (d/f/m)
Inbox

Personio Hiring Team <noreply-recruitment@personio.de>
Wed, Feb 18, 5:10 PM (17 hours ago)
to me

Hi Hayatu,

Thank you once more for your interest in us! You could have applied anywhere, but you chose to pursue a career here at Personio and we really appreciate it.

Unfortunately, we have to inform you that the position Software Engineer - Front End (d/f/m) has now been filled. We know this update comes after some time, and we appreciate the effort you put into your application. While it didn\u2019t work out this time, we\u2019d love to stay in touch for future opportunities that might be a good fit.

Please feel free to check our Career Page in the future for suitable jobs and connect with us on Instagram, LinkedIn & TikTok to stay up to date.

We wish you all the best for your future career!

Warm regards,
Your Talent Team



Personio SE & Co. KG
Seidlstra\u00dfe 3, 80335 Munich
Germany
Commercial Register: HRA 115934, Local Court Munich
VAT-ID DE351718597
General partner: Personio Group SE; Managing Board Member: Hanno Renner; Chair of the Supervisory Board: Roman Schumacher; Commercial Register: HRB 281581, Local Court Munich, Germany

"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)

print(response.choices[0].message.content)